In [1]:
import pandas as pd
import numpy as np
import requests
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

users = pd.read_csv('../dataset/userid-profile.tsv', sep='\t', header=0, index_col='#id')
listening_history = pd.read_csv('../dataset/userid-timestamp-artid-artname-traid-traname.tsv', sep='\t', encoding='utf-8', header=None, error_bad_lines=False)
listening_history.columns = ["#id", "timestamp ", "artist-id", "artist-name", "track-id","track-name"]

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 8709941: expected 6 fields, saw 11\nSkipping line 8709958: expected 6 fields, saw 11\nSkipping line 8710163: expected 6 fields, saw 11\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [26]:
listening_history['tags'] = None
listening_history.head(15)

,#id,timestamp,artist-id,artist-name,track-id,track-name,tags
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,None
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Composition 0919 (Live_2009_4_15),None
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Mc2 (Live_2009_4_15),None
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Hibari (Live_2009_4_15),None
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Mc1 (Live_2009_4_15),None
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,To Stanford (Live_2009_4_15),None
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Improvisation (Live_2009_4_15),None
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Glacier (Live_2009_4_15),None
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Parolibre (Live_2009_4_15),None
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,åæ¬é¾ä¸,NaN,Bibo No Aozora (Live_2009_4_15),None


In [27]:
for index, row in tqdm(listening_history.iterrows(), total=listening_history.shape[0]):
    if type(row['track-id']) == str:
        api = 'http://ws.audioscrobbler.com/2.0/'
        api_key = '05e6fe9ec0ffd5abd2761ab9dbad752c'
        mbid = row['track-id']
        params = {'method':'track.getInfo', 'api_key':api_key, 'mbid':mbid, 'format':'json'}
        r = requests.get(api, params=params)
        r = r.json()
        tags = []
        if 'track' in r:
            for tag in r['track']['toptags']['tag']:
                tags.append(tag['name'])
            if len(tags) > 0:
                listening_history.at[index,'tags'] = tags
#         r = requests.get("http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key=05e6fe9ec0ffd5abd2761ab9dbad752c&mbid=3e92e447-9e1f-440d-bc00-6734469880c5&format=json", verify=False)


KeyboardInterrupt: 